<a href="https://colab.research.google.com/github/mjavadzadeh11/Crawler/blob/master/automate_Tradingview_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup 
import requests 
import pandas as pd
import datetime as dt
import json
import matplotlib.pyplot as plt
#from newspaper import Article
import nltk
nltk.download('punkt')
from textblob import TextBlob
from csv import writer
import time


In [ ]:
url = "https://www.tradingview.com/key-events/?market=crypto"
source = requests.get(url).text
cf = BeautifulSoup(source , 'lxml')

In [ ]:
while True:
  
  try:
    with open('\content\News.csv' , 'a') as f:
      wirter = writer(f)
      for link in cf.find_all('a', class_='cardLink-117psRQF'):
        
        df_dict['Links'].append(link['href'])




      

    
      
      

In [ ]:
  except:
    print('CSV file not found')
    print(' ')
    time.sleep(1)
    print('connecting to tradingviews for collecting news... ')

    #getting urls:
    df_dict={'Symbol':[] , 'Date':[] , 'Headlines':[], 'Desc':[] , 'Links':[] , 'img_url':[]}
    for link in cf.find_all('a', class_='cardLink-117psRQF'):
      df_dict['Links'].append(link['href'])
    #getting img urls:
    for img in cf.find_all('div' , class_='imageWrapper-2Nz2Ij_B ratio2by1-1D2FG29r'):
      df_dict['img_url'].append(img.img['src'])
    #content of News : 
    for content in cf.find_all('div' , class_= 'content-117psRQF'):
      df_dict['Symbol'].append(content.find('span', class_='symbolName-1fSAmU9b').text)
      df_dict['Date'].append(content.find('span',class_='date-1fSAmU9b').text)
      df_dict['Headlines'].append(content.find('div', class_='headline-1fSAmU9b').text)
      df_dict['Desc'].append(content.find('div', class_='description-1fSAmU9b').text)
    


In [ ]:
    #change the type of date in tradingview
    def norm_date(n):
      ''' change trading view date formate to datetime'''
      
      months_dict={'Jan':1, 'Feb':2,'Mar':3,'Apr':4,'May':5, 'Jun':6, 'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12}
      month=n.split(' ')[0]
      month=months_dict[month]
      year=dt.datetime.today().year
      day=int(n.split(' ')[1])
      return dt.date(year,month,day)

    df_dict['Date'] = list(map(norm_date,df_dict['Date']))
    
    df=pd.DataFrame(df_dict)
    df.set_index('Date', inplace=True)

In [ ]:
    def get_polarity(text):
      return TextBlob(text).sentiment.polarity
    title_polarity=df.loc[:,'Headlines'].apply(get_polarity)
    desc_polarity=df.loc[:,'Desc'].apply(get_polarity)
    df['Polarity']=(title_polarity+desc_polarity)/2

In [ ]:
    df.to_csv('News.csv')